In [3]:
import re
import pandas as pd

pth = '../LI22/20220707/dataset.txt'
with open(pth) as f:
    l = f.readlines()
    dataset = pd.DataFrame([list(map(int, re.findall('\d+', line))) for line in l])
dataset.columns = ['label'] + list(range(10))

In [4]:
dataset

,label,0,1,2,3,4,5,6,7,8,9
0,3,205,157,8,45,205,162,19,64,16,50
1,4,197,90,61,23,106,37,31,119,164,45
2,0,114,11,249,231,125,105,232,237,44,1
3,4,220,170,208,30,95,163,35,74,84,235
4,2,88,207,50,178,120,151,54,227,202,96
...,...,...,...,...,...,...,...,...,...,...,...
115,1,206,137,29,165,238,40,142,123,144,214
116,4,40,152,43,168,171,177,30,215,141,95
117,0,118,166,50,249,30,245,28,190,61,89
118,3,26,254,58,13,114,113,76,80,163,29


In [34]:
t = dataset.label
v = dataset.iloc[:, 1:]
t.shape, v.shape

((120,), (120, 10))

In [35]:
from sklearn.model_selection import train_test_split
train_index, test_index = train_test_split(range(len(t)), test_size=0.3, random_state=0)
train_index = sorted(train_index)
test_index = sorted(test_index)
data_train = v.iloc[train_index]
label_train = t.iloc[train_index]
data_test = v.iloc[test_index]
label_test = t.iloc[test_index]
data_train.shape, data_test.shape, label_train.shape, label_test.shape

((84, 10), (36, 10), (84,), (36,))

In [43]:
data_train.head()

,0,1,2,3,4,5,6,7,8,9
0,205,157,8,45,205,162,19,64,16,50
1,197,90,61,23,106,37,31,119,164,45
4,88,207,50,178,120,151,54,227,202,96
5,248,219,146,34,67,25,197,189,87,133
6,215,138,203,38,59,44,116,145,105,127


In [42]:
data_test.head()

,0,1,2,3,4,5,6,7,8,9
2,114,11,249,231,125,105,232,237,44,1
3,220,170,208,30,95,163,35,74,84,235
7,140,153,205,229,108,162,21,163,234,213
8,166,66,77,25,27,188,137,70,133,97
10,106,204,175,16,96,63,196,133,30,20


In [47]:
label_train[:5], label_test[:5]

(0    3
 1    4
 4    2
 5    0
 6    3
 Name: label, dtype: int64,
 2     0
 3     4
 7     4
 8     4
 10    0
 Name: label, dtype: int64)

In [62]:
from sklearn.model_selection import KFold

for i, (train_index, test_index) in enumerate(KFold(n_splits=5, random_state=0, shuffle=True).split(v, t)):
    print(f'(fold=={i})\ntrain_index:\n {train_index}\n test_index:\n {test_index}\n')

(fold==0)
train_index:
 [  0   1   3   4   5   6   9  11  12  13  14  15  17  18  19  20  21  23
  25  26  27  28  29  31  32  34  35  36  37  38  39  40  41  42  44  46
  47  49  51  52  53  54  55  56  57  58  61  62  64  65  66  67  68  69
  70  71  72  73  75  77  78  79  80  81  82  83  84  85  86  87  88  90
  91  92  93  96  98  99 100 101 102 103 104 105 106 107 108 109 110 112
 113 114 115 116 117 118]
 test_index:
 [  2   7   8  10  16  22  24  30  33  43  45  48  50  59  60  63  74  76
  89  94  95  97 111 119]

(fold==1)
train_index:
 [  0   1   2   4   5   7   8   9  10  12  14  15  16  17  19  20  21  22
  23  24  25  28  29  30  31  32  33  34  35  36  37  38  39  40  41  42
  43  44  45  46  47  48  49  50  52  53  55  56  57  58  59  60  62  63
  64  65  67  69  70  71  72  73  74  76  77  78  79  80  81  82  83  85
  86  87  88  89  92  94  95  97  98 101 102 103 105 106 108 109 111 112
 114 115 116 117 118 119]
 test_index:
 [  3   6  11  13  18  26  27  51  54  61  

In [49]:
kf

KFold(n_splits=5, random_state=None, shuffle=False)

In [15]:
import numpy as np

def cos_sim_score(v1, v2):
    return np.dot(v1, v2) / np.linalg.norm(v1) * np.linalg.norm(v2)

def most_sim_vec(v1, vecs_arr):
    sim_scores = [cos_sim_score(v1, v2) for v2 in vecs_arr]
    max_arg = np.argmax(sim_scores)
    max_score = np.max(sim_scores)
    print()
#     max_score == sim_scores[max_arg]
    return max_arg, max_score

In [16]:
vec1 = dataset.iloc[0, 1:]
vecs = dataset.iloc[1:, 1:]
vec1, vecs.shape

(0    205
 1    157
 2      8
 3     45
 4    205
 5    162
 6     19
 7     64
 8     16
 9     50
 Name: 0, dtype: int64,
 (119, 10))

In [17]:
most_sim_vec(vec1, vecs)

IndexError: list index out of range

In [1]:
def computeTF(word_dict, bow):
    tf_dict = {}
    bow_cnt = len(bow)
    for word, cnt in word_dict.items():
        tf_dict[word] = cnt / float(bow_cnt)
    return tf_dict

def computeIDF(doc_dict):
    N = len(doc_dict)
    idf_dict = dict(zip(doc_dict[0].keys(), [0]*len(doc_dict[0])) )
    for doc in doc_dict:
        for word, val in doc.items():
            if val > 0:
                idf_dict[word] += 1
    
    for word, val in idf_dict.items():
        idf_dict[word] = math.log(N / float(val))
    return idf_dict

def computeTFIDF(tf_bow, idfs):
    tfidf = {}
    for word, val in tf_bow.items():
        tfidf[word] = val * idfs[word]
    return tfidf

In [2]:
import glob
import math
def computeTF(word_dict, bow):
    tf_dict = {}
    bow_cnt = len(bow)
    for word, cnt in word_dict.items():
        tf_dict[word] = cnt / float(bow_cnt)
    return tf_dict

def computeIDF(doc_dict):
    N = len(doc_dict)
    idf_dict = dict(zip(doc_dict[0].keys(), [0]*len(doc_dict[0])) )
    for doc in doc_dict:
        for word, val in doc.items():
            if val > 0:
                idf_dict[word] += 1
    
    for word, val in idf_dict.items():
        idf_dict[word] = math.log(N / float(val))
    return idf_dict

def computeTFIDF(tf_bow, idfs):
    tfidf = {}
    for word, val in tf_bow.items():
        tfidf[word] = val * idfs[word]
    return tfidf

def doc2TFIDF(docA, docB):
    # Bag of Words
    bowA = docA.split(' ')
    bowB = docB.split(' ')
    unique = set(bowA + bowB)
    # print(unique)

    num_wordsA = dict(zip(unique, [0]*len(unique)))
    for word in bowA:
        num_wordsA[word] += 1
    num_wordsB = dict(zip(unique, [0]*len(unique)))
    for word in bowB:
        num_wordsB[word] += 1

#     stop_words = stopwords.words('english')

    tfA = computeTF(num_wordsA, bowA)
    tfB = computeTF(num_wordsB, bowB)
    # tfA, tfB

    doc_dict = [num_wordsA, num_wordsB]
    idfs = computeIDF(doc_dict)
    # idfs

    tfidfA = computeTFIDF(tfA, idfs)
    tfidfB = computeTFIDF(tfB, idfs)
    df = pd.DataFrame([tfidfA, tfidfB])

    print(df)

path_list = [fname for fname in glob.glob('../LI22/20220630/data*.txt') if '_' not in fname.split('/')[-1]]
path_list

['../LI22/20220630/data1.txt',
 '../LI22/20220630/data3.txt',
 '../LI22/20220630/data2.txt',
 '../LI22/20220630/data5.txt',
 '../LI22/20220630/data4.txt']

In [3]:
for path in path_list:
    with open(path) as f:
        doc = f.read()
    bag_of_words = ([word.lower() for word in doc.split()])
    unique = set(bag_of_words)
    num_words = dict(zip(unique, [0]*len(unique)))
    for word in bag_of_words:
        num_words[word] += 1

    tf = computeTF(num_words, bag_of_words)
    top5 = sorted([[value, key] for key, value in tf.items()], reverse=True)[:5]
    for i, value_key in enumerate(top5):
        value, key = value_key
        print(f'{i}: {key} {value}')

0: the 0.07348336594911938
1: and 0.05342465753424658
2: of 0.04442270058708415
3: long 0.043835616438356165
4: mike 0.0213307240704501
0: and 0.04199893673577884
1: the 0.03721424774056353
2: to 0.02658160552897395
3: a 0.024455077086656035
4: you 0.018075491759702286
0: the 0.05584192439862543
1: and 0.04338487972508591
2: of 0.03565292096219931
3: in 0.029209621993127148
4: to 0.02491408934707904
0: the 0.06437389770723104
1: to 0.0291005291005291
2: of 0.02601410934744268
3: a 0.020943562610229276
4: and 0.020282186948853614
0: the 0.06012609564816239
1: and 0.02798708288482239
2: of 0.026449331077964017
3: in 0.025988005535906505
4: to 0.0206058742119022


In [23]:
import glob
import math
import pandas as pd
def computeTF(word_dict, bow):
    tf_dict = {}
    bow_cnt = len(bow)
    for word, cnt in word_dict.items():
        tf_dict[word] = cnt / float(bow_cnt)
    return tf_dict

def computeIDF(doc_dict):
    N = len(doc_dict)
    idf_dict = dict(zip(doc_dict[0].keys(), [0]*len(doc_dict[0])) )
    for doc in doc_dict:
        for word, val in doc.items():
            if val > 0:
                idf_dict[word] += 1
    
    for word, val in idf_dict.items():
        idf_dict[word] = math.log(N / float(val))
    return idf_dict

def computeTFIDF(tf_bow, idfs):
    tfidf = {}
    for word, val in tf_bow.items():
        tfidf[word] = val * idfs[word]
    return tfidf

def doc2TFIDF(doc_list, N=5):
    # Bag of Words
#     bow_list = [doc.split(' ') for doc in doc_list]
    bow_list = [[w for w in doc.split(' ') if w != '' and w != '\n'] for doc in doc_list]
#     bow_list = [bow.remove('') if '' in bow else bow for bow in bow_list]
    unique = set(sum(bow_list, []))
#     unique.remove('')
#     bow_list = [word for word in doc.split(' ') for doc in doc_list if word != '' or word != '\n']
#     unique = set(bow_list)

    # print(unique)
    
    list_num_words = [dict(zip(unique, [0]*len(unique))) for i in range(N)]
    for i in range(5):
        for word in bow_list[i]:
            list_num_words[i][word] += 1

    list_tf = [computeTF(num_words, bow) for num_words, bow in zip(list_num_words, bow_list)]

    doc_dict = [num_words for num_words in list_num_words]
    idfs = computeIDF(doc_dict)

    tfidf_list = [computeTFIDF(tf, idfs) for tf in list_tf]
    df_tfidf = pd.DataFrame(tfidf_list)

    return df_tfidf, idfs, unique



path_list = [fname for fname in glob.glob('../LI22/20220630/data*.txt') if '_' not in fname.split('/')[-1]]
doc_list = []
for path in path_list:
    with open(path) as f:
        doc = f.read()
        doc_list.append(doc.lower())
df_tfidf, idfs, unique = doc2TFIDF(doc_list, N=5)
top10s = []
for index, row in df_tfidf.iterrows():
    top10s.extend(sorted([[col, i, index]for i, col in enumerate(row)], reverse=True)[:10])
for score, col, row in sorted(top10s, reverse=True)[:10]:
#     print(score, df.columns[col], f'doc{row}', f'idf = {idfs[df.columns[col]]}') # check idf
    print(score, f'   \'{df_tfidf.columns[col]}\' in docment_{row}')

0.03433047601865302    'mike' in docment_0
0.03007853632826939    'lauren' in docment_0
0.028346264602557538    'demetri' in docment_0
0.02141717769971014    'talk' in docment_0
0.017483042340064602    'morning' in docment_0
0.01385952992408267    'richest' in docment_4
0.013826786189296395    'pharmacy' in docment_2
0.012374580289359529    'almost' in docment_4
0.011752768260901936    'louis' in docment_2
0.011752768260901936    'student' in docment_2


In [18]:
doc_list

[' read the notes wral reporters have made so far in their analysis of the documents click an annotation to read the source document in a new window \n the best of the morning show with mike lauren and demetrihttp toolong tthe best clips and riffs from mike lauren and demetri on the morning show copyright by capitol broadcasting company all rights reserved this material may not be published broadcast rewritten or redistributed wed jun wed jun best of the morning show jun long mike and demetri discuss the th anniversary of the one and done rule how long the panthers will remain on top of the nfc south and the canes schedule for next season wed jun long of the morning show jun long mike demetri and brian discuss peanut tillman what the carolina hurricanes will do in the nhl draft former baylor coach art briles and the brilliance of j r smith tue jun long of the morning show jun long mike and demetri discuss lebron james legacy and give the canes a jun long of the morning show jun long mi

In [21]:
bow_list = [[w for w in doc.split(' ') if w != '' and w != '\n'] for doc in doc_list]

In [22]:
bow_list

[['read',
  'the',
  'notes',
  'wral',
  'reporters',
  'have',
  'made',
  'so',
  'far',
  'in',
  'their',
  'analysis',
  'of',
  'the',
  'documents',
  'click',
  'an',
  'annotation',
  'to',
  'read',
  'the',
  'source',
  'document',
  'in',
  'a',
  'new',
  'window',
  'the',
  'best',
  'of',
  'the',
  'morning',
  'show',
  'with',
  'mike',
  'lauren',
  'and',
  'demetrihttp',
  'toolong',
  'tthe',
  'best',
  'clips',
  'and',
  'riffs',
  'from',
  'mike',
  'lauren',
  'and',
  'demetri',
  'on',
  'the',
  'morning',
  'show',
  'copyright',
  'by',
  'capitol',
  'broadcasting',
  'company',
  'all',
  'rights',
  'reserved',
  'this',
  'material',
  'may',
  'not',
  'be',
  'published',
  'broadcast',
  'rewritten',
  'or',
  'redistributed',
  'wed',
  'jun',
  'wed',
  'jun',
  'best',
  'of',
  'the',
  'morning',
  'show',
  'jun',
  'long',
  'mike',
  'and',
  'demetri',
  'discuss',
  'the',
  'th',
  'anniversary',
  'of',
  'the',
  'one',
  'and',
 

In [119]:
idfs['mike'], idfs['lauren']

(1.6094379124341003, 1.6094379124341003)

In [121]:
math.log(5 / 2), math.log(5 / 1)

(0.9162907318741551, 1.6094379124341003)